In [30]:
%pip install alpaca-trade-api
%pip install alpaca-py

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: celery 5.1.2 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: celery 5.1.2 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a co

In [31]:
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca_trade_api.rest import REST, TimeFrame
import pandas as pd

# No keys required for crypto data
client = CryptoHistoricalDataClient()

In [32]:
import os
from dotenv import load_dotenv
load_dotenv()
BASE_URL = os.getenv("BASE_URL")
KEY_ID = os.getenv("KEY_ID")
SECRET_KEY = os.getenv("SECRET_KEY")
# Instantiate REST API Connection
api = REST(key_id=KEY_ID,secret_key=SECRET_KEY,base_url="https://paper-api.alpaca.markets")

# Fetch 1Minute historical bars of Ether
bars = api.get_crypto_bars("ETH/USD", TimeFrame.Minute).df
# bars = bars[bars.exchange == 'CBSE']
print(bars)


                               close       high        low  trade_count  \
timestamp                                                                 
2024-08-23 05:00:00+00:00  2638.2900  2638.2900  2638.2900            0   
2024-08-23 05:01:00+00:00  2641.6795  2641.6795  2641.6795            0   
2024-08-23 05:02:00+00:00  2645.9050  2645.9050  2643.6245            0   
2024-08-23 05:07:00+00:00  2645.4020  2645.4020  2645.4020            0   
2024-08-23 05:09:00+00:00  2645.6925  2645.6925  2645.6925            0   
...                              ...        ...        ...          ...   
2024-08-23 19:59:00+00:00  2747.5550  2747.5550  2747.5550            0   
2024-08-23 20:00:00+00:00  2749.6100  2749.6100  2749.6100            0   
2024-08-23 20:01:00+00:00  2748.7600  2748.7600  2748.7600            0   
2024-08-23 20:02:00+00:00  2748.1505  2748.7600  2747.6055            0   
2024-08-23 20:03:00+00:00  2747.6450  2747.6450  2747.6450            0   

                        

In [18]:
# Create a market order to buy 1 Eth
order_buy = api.submit_order('ETH/USD', qty=1, side='buy', time_in_force = "gtc")
#gtc is the only valid one for for crypto, why

In [34]:
from datetime import datetime, timedelta
import math
import time

def get_position(symbol):
    positions = api.list_positions()
    for p in positions:
        if p.symbol == symbol:
            return float(p.qty)
    return 0

SYMBOL = 'ETH/USD'
SMA_FAST = 12
SMA_SLOW = 24
QTY_PER_TRADE = 1



def get_pause():
    now = datetime.now()
    next_min = now.replace(second=0, microsecond=0) + timedelta(minutes=1)
    pause = math.ceil((next_min - now).seconds)
    print(f"Sleep for {pause}")
    return pause


def get_position(symbol):
    positions = api.list_positions()
    for p in positions:
        if p.symbol == symbol:
            return float(p.qty)
    return 0


# Returns a series with the moving average
def get_sma(series, periods):
    return series.rolling(periods).mean()

# Checks wether we should buy (fast ma > slow ma)
def get_signal(fast, slow):
    print(f"Fast {fast[-1]}  /  Slow: {slow[-1]}")
    return fast[-1] > slow[-1]

# Get up-to-date 1 minute data from Alpaca and add the moving averages
def get_bars(symbol):
    bars = api.get_crypto_bars(symbol, TimeFrame.Minute).df
    bars[f'sma_fast'] = get_sma(bars.close, SMA_FAST)
    bars[f'sma_slow'] = get_sma(bars.close, SMA_SLOW)
    return bars

while True:
    # GET DATA
    bars = get_bars(symbol=SYMBOL)
    # CHECK POSITIONS
    position = get_position(symbol=SYMBOL)
    should_buy = get_signal(bars.sma_fast,bars.sma_slow)
    print(f"Position: {position} / Should Buy: {should_buy}")
    if position == 0 and should_buy == True:
        # WE BUY ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='buy', time_in_force="gtc")
        print(f'Symbol: {SYMBOL} / Side: BUY / Quantity: {QTY_PER_TRADE}')
    elif position > 0 and should_buy == False:
        # WE SELL ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='sell')
        print(f'Symbol: {SYMBOL} / Side: SELL / Quantity: {QTY_PER_TRADE}')

    time.sleep(get_pause())
    print("*"*20)

/var/folders/sn/wcs7_4d549z9dj_nct3c3y5m0000gn/T/ipykernel_59314/2013956955.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Fast {fast[-1]}  /  Slow: {slow[-1]}")
/var/folders/sn/wcs7_4d549z9dj_nct3c3y5m0000gn/T/ipykernel_59314/2013956955.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return fast[-1] > slow[-1]


Fast 2747.426  /  Slow: 2745.0944375
Position: 0 / Should Buy: True
Symbol: ETH/USD / Side: BUY / Quantity: 1
Sleep for 39
********************


/var/folders/sn/wcs7_4d549z9dj_nct3c3y5m0000gn/T/ipykernel_59314/2013956955.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Fast {fast[-1]}  /  Slow: {slow[-1]}")
/var/folders/sn/wcs7_4d549z9dj_nct3c3y5m0000gn/T/ipykernel_59314/2013956955.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return fast[-1] > slow[-1]


Fast 2748.115916666667  /  Slow: 2745.3654375
Position: 0 / Should Buy: True
Symbol: ETH/USD / Side: BUY / Quantity: 1
Sleep for 0
********************


/var/folders/sn/wcs7_4d549z9dj_nct3c3y5m0000gn/T/ipykernel_59314/2013956955.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Fast {fast[-1]}  /  Slow: {slow[-1]}")
/var/folders/sn/wcs7_4d549z9dj_nct3c3y5m0000gn/T/ipykernel_59314/2013956955.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return fast[-1] > slow[-1]


...

Fast 2753.677  /  Slow: 2749.9753333333333
Position: 0 / Should Buy: True
Symbol: ETH/USD / Side: BUY / Quantity: 1
Sleep for 59
********************


/var/folders/sn/wcs7_4d549z9dj_nct3c3y5m0000gn/T/ipykernel_59314/2013956955.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Fast {fast[-1]}  /  Slow: {slow[-1]}")
/var/folders/sn/wcs7_4d549z9dj_nct3c3y5m0000gn/T/ipykernel_59314/2013956955.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return fast[-1] > slow[-1]


Fast 2754.61725  /  Slow: 2750.7147499999996
Position: 0 / Should Buy: True
Symbol: ETH/USD / Side: BUY / Quantity: 1
Sleep for 0
********************
Fast 2754.61725  /  Slow: 2750.7147499999996
Position: 0 / Should Buy: True


/var/folders/sn/wcs7_4d549z9dj_nct3c3y5m0000gn/T/ipykernel_59314/2013956955.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Fast {fast[-1]}  /  Slow: {slow[-1]}")
/var/folders/sn/wcs7_4d549z9dj_nct3c3y5m0000gn/T/ipykernel_59314/2013956955.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return fast[-1] > slow[-1]


APIError: insufficient balance for USD (requested: 2814.38, available: 816.55)